# **Use state of art libraries for Approximate nearest neighbor search for your favorite data set**

a) LSH

b) exhaustive search

c) product quantization

d) trees and graphs

e) hnsw

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
!python -m pip install --upgrade faiss faiss-gpu
!pip install datasketch
!pip install sentence_transformers
!pip install hnswlib
!pip install nmslib
!pip install annoy

     |████████████████████████████████| 646 kB 7.9 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391670 sha256=71083ca01ee59476ea9b03daf4437bbb6c4c987294b6f08b3c6066a817402cab
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [39]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sn
import math
import warnings # Current version of Seaborn generates a bunch of warnings that will be ignored.
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import nmslib
import hnswlib

I will be using a dataset of movies pulled from Wikipedia. The original Dataset can be found on Kaggle at https://www.kaggle.com/jrobischon/wikipedia-movie-plots?select=wiki_movie_plots_deduped.csv.

In [5]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/wiki_movie_plots_deduped.csv')
df.head(10)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab..."
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....


In [6]:
df.drop(columns=["Origin/Ethnicity", "Cast", "Genre"], inplace= True)
df["text"] = df["Title"] + " " + df["Plot"]
df.head(10)

,Release Year,Title,Director,Wiki Page,Plot,text
0,1901,Kansas Saloon Smashers,Unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers A bartender is working ...
1,1901,Love by the Light of the Moon,Unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon The moon, painte..."
2,1901,The Martyred Presidents,Unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The Martyred Presidents The film, just over a ..."
3,1901,"Terrible Teddy, the Grizzly King",Unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King Lasting just ..."
4,1902,Jack and the Beanstalk,"George S. Fleming, Edwin S. Porter",https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,Jack and the Beanstalk The earliest known adap...
5,1903,Alice in Wonderland,Cecil Hepworth,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland Alice follows a large whit...
6,1903,The Great Train Robbery,Edwin S. Porter,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,The Great Train Robbery The film opens with tw...
7,1904,The Suburbanite,Wallace McCutcheon,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,The Suburbanite The film is about a family who...
8,1905,The Little Train Robbery,Edwin Stanton Porter,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,The Little Train Robbery The opening scene sho...
9,1905,The Night Before Christmas,Edwin Stanton Porter,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,The Night Before Christmas Scenes are introduc...


Get first 1000 rows for performance purposes, otherwise might run into RAM issues on colab.

In [7]:
df = df[0:1000]
df.shape

(1000, 6)

# **LSH**

In [8]:
import re
import time
from datasketch import MinHash, MinHashLSHForest

Preprocess the data by creating the function for tokenizing the text. This removes stop words and lemmatizes the text.

In [9]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

Pass the columns of text and minhash using the MinHash and MinHashLHSForest functions.

In [10]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

Process text with shingles and query for recommended results based on similarity with movie query.

In [11]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['Title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [12]:
permutations = 128

num_recommendations = 1

Create the forest, and then in title enter a movie title and it should return recommendations of similar movies.

In [13]:
forest = get_forest(df, permutations)

It took 4.729840278625488 seconds to build forest.


In [14]:
num_recommendations = 5
title = 'The Godfather'
result = predict(title, df, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.005959987640380859 seconds to query forest.

 Top Recommendation(s) is(are) 
 25      The Lure of the Gown
193    Are Crooks Dishonest?
Name: Title, dtype: object


# **Exhaustive Search**

In [15]:
import faiss

In [16]:
df['Index'] = df.index

Vectorize the data using TFidfVectorizer.

In [17]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0,stop_words='english')
X_tfidf = tfidf.fit_transform(df['text'])
X_tfidf

<1000x228586 sparse matrix of type '<class 'numpy.float64'>'
	with 307021 stored elements in Compressed Sparse Row format>

In [18]:
vector = X_tfidf.toarray()
vector[0].shape[0]

228586

Create a function using faiss.IndexFlatL2 for exhaustive search, and then run the function over the vectorized text data.

In [19]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [20]:
%%time
index = ExactIndex(vector, df["Index"])
index.build()

CPU times: user 659 ms, sys: 988 ms, total: 1.65 s
Wall time: 1.65 s


Give a query index and it should return the most similar movies the the query given.

In [21]:
query_index = 5
print(f"The most similar items to item {df['Index'][query_index]} are:")

items_list = index.query(np.array([vector[query_index]]).astype('float32'))
df.loc[df['Index'].isin(items_list)]

The most similar items to item 5 are:


,Release Year,Title,Director,Wiki Page,Plot,text,Index
5,1903,Alice in Wonderland,Cecil Hepworth,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland Alice follows a large whit...,5
150,1916,Sherlock Holmes,Arthur Berthelet,https://en.wikipedia.org/wiki/Sherlock_Holmes_...,"A prince, the heir apparent to a large empire,...","Sherlock Holmes A prince, the heir apparent to...",150
170,1917,His Wedding Night,Roscoe Arbuckle,https://en.wikipedia.org/wiki/His_Wedding_Night,"Arbuckle plays a drug store clerk, soda jerk, ...",His Wedding Night Arbuckle plays a drug store ...,170
225,1918,Moonshine,Unknown,https://en.wikipedia.org/wiki/Moonshine_(1918_...,"Set in the Virginia Hills, Fatty and Buster pl...","Moonshine Set in the Virginia Hills, Fatty and...",225
495,1924,Helen's Babies,William A. Seiter,https://en.wikipedia.org/wiki/Helen%27s_Babies...,Toodie and Budge are identified as the two bes...,Helen's Babies Toodie and Budge are identified...,495
526,1925,Confessions of a Queen,Victor Sjostrom,https://en.wikipedia.org/wiki/Confessions_of_a...,The King of Illyris (Lewis Stone) marries a ne...,Confessions of a Queen The King of Illyris (Le...,526
895,1930,Borrowed Wives,Frank R. Strayer,https://en.wikipedia.org/wiki/Borrowed_Wives,Peter Foley (Rex Lease) is a beneficiary of hi...,Borrowed Wives Peter Foley (Rex Lease) is a be...,895
917,1930,Extravagance,Phil Rosen,https://en.wikipedia.org/wiki/Extravagance_(film),Alice Kendall is the darling of her social set...,Extravagance Alice Kendall is the darling of h...,917
935,1930,He Knew Women,Hugh Herbert,https://en.wikipedia.org/wiki/He_Knew_Women,"Geoffrey Clarke is a poor poet, who has his ey...","He Knew Women Geoffrey Clarke is a poor poet, ...",935
995,1930,Playing Around,Mervyn LeRoy,https://en.wikipedia.org/wiki/Playing_Around,Alice White plays the part of a working class ...,Playing Around Alice White plays the part of a...,995


# **Product Quantization**

In [28]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Embed the text using normalizeL2.

In [42]:
%%time
embeddings = model.encode(df['text'].tolist())
faiss.normalize_L2(embeddings)

CPU times: user 3min 1s, sys: 863 ms, total: 3min 1s
Wall time: 3min


In [30]:
print("Shape of the Embeddings is ",embeddings.shape)

Shape of the Embeddings is  (1000, 768)


Use the IndexFlatL2 and IndexIVFPQ Quantization models to do product quantization on the text data.

In [32]:
dim=768
ncentroids=50
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, 16 , 8)
index.train(embeddings) 
print(index.is_trained)
faiss.write_index(index, "trained.index")

True


In [33]:
ids = df['Index'].tolist()
ids = np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

1000


In [34]:
faiss.write_index(index,"block.index")

In [35]:
def searchFAISSIndex(data, id_col_name, query, index, nprobe, model, topk=20):
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)
  
    index.nprobe=nprobe
    
    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [36]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

Give a query and it should give the top recommended movies matching the queries (in this case the top 20 and the cosine_sim and L2 scores.

In [40]:
query = "A seventeen-year-old aristocrat falls in love with a kind but poor artist"
search_result = searchFAISSIndex(df, "Index", query, index, nprobe=10, model= model, topk=20)
search_result = search_result[['Index', 'Title', 'Plot', 'cosine_sim', 'L2_score']]

In [41]:
search_result

,Index,Title,Plot,cosine_sim,L2_score
14,686,Dream of Love,"Adrienne, a Gypsy girl performing in a traveli...",0.627798,0.862788
15,691,Forbidden Hours,Set in the fictitious European kingdom of Bala...,0.561787,0.936176
5,467,Merry-Go-Round,"A nobleman, posing as a necktie salesman, fall...",0.558208,0.939991
2,384,Heedless Moths,"As described in a film publication,[3] idealis...",0.516007,0.983863
7,526,Confessions of a Queen,The King of Illyris (Lewis Stone) marries a ne...,0.515937,0.983934
3,460,Black Oxen,"Lee Clavering (Tearle), a playwright in New Yo...",0.503277,0.996717
13,682,The Divine Woman,Marianne (Greta Garbo) is a poor French countr...,0.493323,1.006655
4,465,Gentle Julia,"Julia (Bessie Love) is a ""small-town heartbrea...",0.487215,1.012704
17,821,Married in Hollywood,"A showgirl, part of a troupe, tours Europe whe...",0.485977,1.013926
8,537,The King on Main Street,King Serge IV of Molvania (Menjou) comes to a ...,0.478432,1.021340


# **Trees and Graphs**

In [56]:
import nltk
from nltk.corpus import wordnet as wn
import annoy

Create Graph using AnnoyIndex and the run the function over the tokenized vector I created using TFidf earlier.

In [57]:
class TreeGraphIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention, metric='angular')
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [59]:
%%time
tg_index = TreeGraphIndex(vector, df['Index'])
tg_index.build()

CPU times: user 15.7 s, sys: 6.51 s, total: 22.2 s
Wall time: 22.2 s


Query using a query index and it should give the most similar movies to the queried movie.

In [61]:
query_index = 5
print(f"The most similar items to item {df['Index'][query_index]} are:")
items_list = tg_index.query(vector[query_index])
df.loc[df['Index'].isin(items_list)]

The most similar items to item 5 are:


,Release Year,Title,Director,Wiki Page,Plot,text,Index
5,1903,Alice in Wonderland,Cecil Hepworth,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland Alice follows a large whit...,5
150,1916,Sherlock Holmes,Arthur Berthelet,https://en.wikipedia.org/wiki/Sherlock_Holmes_...,"A prince, the heir apparent to a large empire,...","Sherlock Holmes A prince, the heir apparent to...",150
170,1917,His Wedding Night,Roscoe Arbuckle,https://en.wikipedia.org/wiki/His_Wedding_Night,"Arbuckle plays a drug store clerk, soda jerk, ...",His Wedding Night Arbuckle plays a drug store ...,170
225,1918,Moonshine,Unknown,https://en.wikipedia.org/wiki/Moonshine_(1918_...,"Set in the Virginia Hills, Fatty and Buster pl...","Moonshine Set in the Virginia Hills, Fatty and...",225
495,1924,Helen's Babies,William A. Seiter,https://en.wikipedia.org/wiki/Helen%27s_Babies...,Toodie and Budge are identified as the two bes...,Helen's Babies Toodie and Budge are identified...,495
526,1925,Confessions of a Queen,Victor Sjostrom,https://en.wikipedia.org/wiki/Confessions_of_a...,The King of Illyris (Lewis Stone) marries a ne...,Confessions of a Queen The King of Illyris (Le...,526
895,1930,Borrowed Wives,Frank R. Strayer,https://en.wikipedia.org/wiki/Borrowed_Wives,Peter Foley (Rex Lease) is a beneficiary of hi...,Borrowed Wives Peter Foley (Rex Lease) is a be...,895
917,1930,Extravagance,Phil Rosen,https://en.wikipedia.org/wiki/Extravagance_(film),Alice Kendall is the darling of her social set...,Extravagance Alice Kendall is the darling of h...,917
935,1930,He Knew Women,Hugh Herbert,https://en.wikipedia.org/wiki/He_Knew_Women,"Geoffrey Clarke is a poor poet, who has his ey...","He Knew Women Geoffrey Clarke is a poor poet, ...",935
995,1930,Playing Around,Mervyn LeRoy,https://en.wikipedia.org/wiki/Playing_Around,Alice White plays the part of a working class ...,Playing Around Alice White plays the part of a...,995


# **HSNW**

Initialize a new index, using a HNSW index on Cosine Similarity on the tokenized vector I made earlier.

In [46]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(vector)
index.createIndex({'post': 2}, print_progress=True)

In [48]:
ids, distances = index.knnQuery(vector[0], k=10)

Query for the nearest neighbours of the first datapoint

In [49]:
neighbours = index.knnQueryBatch(vector, k=10, num_threads=4)

Get all nearest neighbours for all the movies using a pool of 4 threads to compute

In [50]:
neighbours

[(array([  0, 231, 669, 731,  21, 170, 459, 166, 283, 611], dtype=int32),
  array([6.5565109e-07, 9.5172840e-01, 9.8116642e-01, 9.8174721e-01,
         9.8481107e-01, 9.8512775e-01, 9.8533785e-01, 9.8644912e-01,
         9.8674852e-01, 9.8689181e-01], dtype=float32)),
 (array([  1, 696, 997, 182,  69, 102, 974, 745, 400, 340], dtype=int32),
  array([1.2516975e-06, 9.4462037e-01, 9.5570296e-01, 9.7994041e-01,
         9.8087382e-01, 9.8308253e-01, 9.8329198e-01, 9.8701847e-01,
         9.8781264e-01, 9.8781264e-01], dtype=float32)),
 (array([  2, 880,   3, 130, 598,  22,  60, 101, 315, 903], dtype=int32),
  array([0.        , 0.9575359 , 0.9699915 , 0.97803646, 0.9790967 ,
         0.98064625, 0.98412555, 0.9846031 , 0.98557943, 0.9862399 ],
        dtype=float32)),
 (array([  3, 861,   2,  78, 945, 263, 744, 987,  97, 803], dtype=int32),
  array([0.        , 0.9602857 , 0.9699915 , 0.97464395, 0.97607183,
         0.9781251 , 0.9795348 , 0.9806474 , 0.98364055, 0.98385   ],
        dty

# **Conclusion**
LSH and Exhaustive search were slower than HSNW and Product Quantization, making HSNW and Product Quantization better options for conducting movie similarity recommendations on this specific dataset.


# **References**



1.   https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/
2. https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6
3. https://nmslib.github.io/nmslib/quickstart.html

